In [1]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import os
import json

username = "monikit-student"
## project base path
# base_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
base_path = f"/home/{username}/thesis/CosDefence/"

In [29]:
def compare_histo_plots(trust_df1, trust_df2, overlap=False):
    if overlap:
        pass
    else:
        histo_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01)
        histo_fig.add_trace(go.histogram(trust_df1, x="modified_trust", color=trust_df1["client_label"], color_discrete_map={
                "honest": '#00CC96',
                "minor_offender": '#EF553B',
                "major_offender": '#AB63FA'}), row=1, col=1)
        histo_fig.add_trace(go.histogram(trust_df2, x=trust_df2["modified_trust", color="client_label", color_discrete_map={
                "honest": '#00CC96',
                "minor_offender": '#EF553B',
                "major_offender": '#AB63FA'}), row=1, col=1)
        histo_fig.update_layout(title='Trust given by validation clients', barmode="group")
        histo_fig.show()


def compare_trust_curve_plots(trust_score_df1, trust_score_df2):
    pass


def compare_acc_poison_plots(acc_poison_df1, acc_poison_df2, overlap=False):
    pass

In [30]:
trust_df1 = pd.read_pickle(f"{base_path}results/plot_dfs/trust_fmnist_C0.2_P0.6_FDRS60_CDFTrue_SEL1_CLBTrue_LYRSl1_FFAauror_plus_CSEP2.0_2021-09-02 16:53:12.pkl")
trust_df2 = pd.read_pickle(f"{base_path}results/plot_dfs/trust_fmnist_C0.2_P0.4_FDRS60_CDFTrue_SEL1_CLBTrue_LYRSl1_FFAauror_plus_CSEP2.0_2021-09-02 10:51:17.pkl")

# print(trust_df1.head())
# print(trust_df2.head())
compare_histo_plots(trust_df1, trust_df2)

In [26]:
x = ['1970-01-01', '1970-01-01', '1970-02-01', '1970-04-01', '1970-01-02',
     '1972-01-31', '1970-02-13', '1971-04-19']

fig = make_subplots(rows=3, cols=2)

trace0 = go.Histogram(x=x, nbinsx=4)
trace1 = go.Histogram(x=x, nbinsx = 8)
trace2 = go.Histogram(x=x, nbinsx=10)
trace3 = go.Histogram(x=x,
                      xbins=dict(
                      start='1969-11-15',
                      end='1972-03-31',
                      size='M18'), # M18 stands for 18 months
                      autobinx=False
                     )
trace4 = go.Histogram(x=x,
                      xbins=dict(
                      start='1969-11-15',
                      end='1972-03-31',
                      size='M4'), # 4 months bin size
                      autobinx=False
                      )
trace5 = go.Histogram(x=x,
                      xbins=dict(
                      start='1969-11-15',
                      end='1972-03-31',
                      size= 'M2'), # 2 months
                      autobinx = False
                      )

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)
fig.append_trace(trace4, 3, 1)
fig.append_trace(trace5, 3, 2)

fig.show()

## Comparing Cos_defence performance with different poison_frac vals 

In [38]:
def get_parsed_summary_data(summary_file):
    p_fracs = list()
    on_mm_poison_class_acc = list()
    off_mm_poison_class_acc = list()
    
    with open(summary_file) as json_file:
        summary_data = json.load(json_file)
        for summary in summary_data['summary']:
            on_summary = summary['cdf_on_summary']
            p_fracs.append(on_summary['config']['POISON_FRAC'])
            on_mm_poison_class_acc.append(on_summary['mean_mean_poison_class_accs_on'])
            
            off_summary = summary['cdf_off_summary']
            off_mm_poison_class_acc.append(off_summary['mean_mean_poison_class_accs_off'])
    
    on_df = pd.DataFrame()
    on_df['p_frac'] = p_fracs
    on_df['mean_mean_poison_class_acc'] = on_mm_poison_class_acc
    
    off_df = pd.DataFrame()
    off_df['p_frac'] = p_fracs
    off_df['mean_mean_poison_class_acc'] = off_mm_poison_class_acc
    
    return on_df, off_df

In [43]:
def plot_graph(on_df, off_df, dataset_name):
    
    # Create traces
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=on_df['p_frac'][1:], y=on_df['mean_mean_poison_class_acc'][1:],
                        mode='lines + markers',
                        name='cos_defence on'))
    fig.add_trace(go.Scatter(x=off_df['p_frac'][1:], y=off_df['mean_mean_poison_class_acc'][1:],
                        mode='lines + markers',
                        name='cos_defence off'))
    
    fig.update_layout(title_text=f"{dataset_name} Poison Class Accuracy plot with increasing poisoned client fraction, 0.2 frac client selected in each round")

    fig.show()

    
def plot_data(summary_file, dataset_name):
    off_df, on_df = get_parsed_summary_data(summary_file)
    plot_graph(off_df, on_df, dataset_name)


data_path = "/home/fnx11/remote_results/"
mnist_file = data_path + "desktop/json_files/summary_mnist_C0.2_FDRS30_{'2021-09-09 18:03:12'}.json"
fmnist_file = data_path + "desktop/json_files/summary_fmnist_C0.2_FDRS60_{'2021-09-09 02:13:19'}.json"
cifar10_file = data_path + "rwth_my/json_files/summary_cifar10_C0.2_FDRS200_{'2021-09-09 00:25:21'}.json"

plot_data(cifar10_file, "cifar10")

In [44]:
plot_data(fmnist_file, "fmnist")

In [45]:
plot_data(mnist_file, "mnist")